#### Find smaller model for classification.
#### Think about generating dataset for fine-tunning

In [1]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="key",
    secret_key="key",
    host="https://cloud.langfuse.com"
)

In [2]:
from langchain_community.chat_models import ChatLlamaCpp 
from langchain_community.llms import LlamaCpp
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Path to the model weights
models_directory = '/home/arseniy/Документы/Projects/Python/LLM_project/models/finetunned_models/'

#local_model = models_directory + 'qwen2-0_5b-instruct-q8_0.gguf' 

#local_model = models_directory + 'qwen2-1_5b-instruct-q4_k_m.gguf'

#local_model = models_directory + 'Qwen2-tunned-100k.gguf' 
#local_model = models_directory + 'Qwen2-Instruct-tunned-320k.gguf'

local_model = models_directory + 'Qwen2-0.5b-tunned-3K.gguf'

llm = ChatLlamaCpp(
    temperature = 0.3,
    #cache = True,
    model_path = local_model,
    n_ctx=3500,
    top_k = 0.5,
    top_p = 0.3,
    n_gpu_layers=33,
    #use_mlock = True,
    # repeat_penalty = 1.6,
    versbose = True,
    callback_manager=callback_manager, 

)


llama_model_loader: loaded meta data with 26 key-value pairs and 290 tensors from /home/arseniy/Документы/Projects/Python/LLM_project/models/finetunned_models/Qwen2-0.5b-tunned-3K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen2 0.5B Instruct
llama_model_loader: - kv   3:                       general.organization str              = Unsloth
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Qwen2
llama_model_loader: - kv   6:                         general.size_label str       

In [3]:
with open('/home/arseniy/Документы/Projects/Python/LLM_project/db_files/mys_schema.txt', 'r') as file:
    schema = file.read()

In [4]:
question = ""


template = """
Given the following database schema:

### Database schema:
{schema}

Please identify the names of the relevant tables that can be used to query the database based on the following user question:

### User question:
"{prompt}"

Respond with only the names of the relevant tables. Take the table name after "CREATE TABLE" statement. Do not write anything else. If the prompt involves multiple tables, list all the relevant table names separated by commas. List all related table names even if you thin that they are not so related to the prompt.

### Relevant Tables:
"""


# Initialize the prompt template with placeholders for 'schema' and 'question'
#prompt_template = PromptTemplate.from_template(sql_prompt_template)
prompt_template = PromptTemplate.from_template(template)

In [5]:
chain = LLMChain(prompt=prompt_template, llm=llm)

/home/arseniy/.var/app/org.jupyter.JupyterLab/config/jupyterlab-desktop/jlab_server/envs/llamaCuda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [6]:
auto_test = True

In [7]:
import sqlvalidator
import sqlcorrect as sc
import finetunning_test as ft

if auto_test:
    ft.llm_test (chain, schema)

else:
    def main():       
        print("🔴 Simple database chatbot app. Type 'exit' to quit.")
        while True:
            # Get user input
            question = input("🟢 Enter your question: ")
            if question.lower() == 'exit':
                print("Exiting the app.")
                break 
    
            #response = chain.run(schema=description, prompt = question)
            #response = chain.invoke({"schema": schema, "prompt": question},config={"callbacks": [langfuse_handler]})
            response = chain.invoke({"schema": schema, "prompt": question},config={"callbacks": [langfuse_handler]})
    
    if __name__ == "__main__":
        main()

employees, certified, project management


llama_print_timings:        load time =     361.29 ms
llama_print_timings:      sample time =      75.70 ms /     7 runs   (   10.81 ms per token,    92.47 tokens per second)
llama_print_timings: prompt eval time =   23884.63 ms /  2366 tokens (   10.09 ms per token,    99.06 tokens per second)
llama_print_timings:        eval time =     322.14 ms /     6 runs   (   53.69 ms per token,    18.63 tokens per second)
llama_print_timings:       total time =   25521.27 ms /  2372 tokens


1. Which employees are certified in project management? ❌
Model answer: employees, certified, project management
Correct answer: employees, certification


Llama.generate: prefix-match hit


employees, project

...

31. List employees who have 'Artificial Intelligence' skills and have worked on 'Machine Learning' projects. ❌
Model answer: employees, skills, projects
Correct answer: employees, softwares, mysProjects


Llama.generate: prefix-match hit


employees, softwares


llama_print_timings:        load time =     361.29 ms
llama_print_timings:      sample time =      55.89 ms /     5 runs   (   11.18 ms per token,    89.46 tokens per second)
llama_print_timings: prompt eval time =    1075.17 ms /    90 tokens (   11.95 ms per token,    83.71 tokens per second)
llama_print_timings:        eval time =     183.25 ms /     4 runs   (   45.81 ms per token,    21.83 tokens per second)
llama_print_timings:       total time =    1324.17 ms /    94 tokens


32. Find employees who are fluent in 'Korean' and have 'Data Science' certifications. ❌
Model answer: employees, softwares
Correct answer: employees, languages, certification

🔴 Model score: 0/32 answers.
